In [1]:
import numpy as np
import random
import math

class MutationOperator:
    def __init__(self, name):
        self.name = name
        self.utility = 0.0  # Espérance d'amélioration
        self.count = 0      # Nombre de sélections

    def mutate(self, solution):
        """Applique l'opérateur de mutation à la solution donnée."""
        mutated = solution.copy()
        if self.name == 'mutate1flip':
            idx = random.randint(0, len(solution) - 1)
            mutated[idx] = 1 - mutated[idx]
        elif self.name == 'mutateBflip':
            p = 1.0 / len(solution)  # Probabilité de flip
            for i in range(len(solution)):
                if random.random() < p:
                    mutated[i] = 1 - mutated[i]
        elif self.name == 'mutate3flip':
            indices = random.sample(range(len(solution)), 3)
            for idx in indices:
                mutated[idx] = 1 - mutated[idx]
        elif self.name == 'mutate5flip':
            indices = random.sample(range(len(solution)), 5)
            for idx in indices:
                mutated[idx] = 1 - mutated[idx]
        else:
            raise ValueError("Opérateur de mutation inconnu.")
        return mutated

class UCB1Policy:
    def __init__(self, operators, p_min=1e-4):
        self.operators = operators
        self.total_counts = 0
        self.p_min = p_min

    def select_operator(self):
        """Sélectionne un opérateur basé sur la stratégie UCB1."""
        # Initialisation : jouer chaque bras au moins une fois
        for op in self.operators:
            if op.count == 0:
                return op

        # Calculer UCB1 pour chaque opérateur
        ucb_values = []
        for op in self.operators:
            average_reward = op.utility / op.count
            confidence = math.sqrt(2 * math.log(self.total_counts) / op.count)
            ucb = average_reward + confidence
            ucb_values.append((ucb, op))
        
        # Sélectionner l'opérateur avec le UCB le plus élevé
        selected_op = max(ucb_values, key=lambda x: x[0])[1]
        return selected_op

    def update(self, operator, reward):
        """Met à jour l'utilité et le compte de l'opérateur sélectionné."""
        operator.count += 1
        operator.utility += reward
        if operator.count < 0:
            operator.count = 0
        self.total_counts += 1

def initialize_solution(length):
    """Initialise une solution binaire aléatoire."""
    return [random.randint(0, 1) for _ in range(length)]

def evaluate(solution):
    """Évalue la solution en comptant le nombre de 1."""
    return sum(solution)

def one_max_algorithm(
    bit_length=100,
    generations=1000,
    operators_names=['mutate1flip', 'mutateBflip', 'mutate3flip', 'mutate5flip']
):
    # Initialisation
    current_solution = initialize_solution(bit_length)
    current_score = evaluate(current_solution)
    best_solution = current_solution.copy()
    best_score = current_score

    # Initialiser les opérateurs de mutation
    operators = [MutationOperator(name) for name in operators_names]
    policy = UCB1Policy(operators)

    for gen in range(1, generations + 1):
        # Sélectionner un opérateur de mutation
        operator = policy.select_operator()

        # Appliquer l'opérateur de mutation
        new_solution = operator.mutate(current_solution)
        new_score = evaluate(new_solution)

        # Calculer le gain (amélioration)
        gain = new_score - current_score

        # Décider d'accepter ou non la nouvelle solution
        if gain > 0:
            current_solution = new_solution
            current_score = new_score
            if current_score > best_score:
                best_solution = current_solution.copy()
                best_score = current_score

        # Mettre à jour la politique avec le gain
        policy.update(operator, gain)

        # Affichage périodique
        if gen % 100 == 0 or best_score == bit_length:
            print(f"Génération {gen}: Meilleur score = {best_score}")

        # Terminer si la solution optimale est trouvée
        if best_score == bit_length:
            print("Solution optimale trouvée.")
            break

    return best_solution, best_score

if __name__ == "__main__":
    best_sol, best_sc = one_max_algorithm()
    print(f"Meilleure solution trouvée: {best_sol}")
    print(f"Nombre de 1s: {best_sc}")


Génération 100: Meilleur score = 80
Génération 200: Meilleur score = 87
Génération 300: Meilleur score = 95
Génération 400: Meilleur score = 96
Génération 500: Meilleur score = 99
Génération 532: Meilleur score = 100
Solution optimale trouvée.
Meilleure solution trouvée: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Nombre de 1s: 100
